In [1]:
# 1. 0.27
# 2 5
# 3 Line is x2 > 6 == 0
# 4) MLE may not exist, non-negative, local is global

# Later parts
#1) Does bfgs work?


# [a1] =
# [b1] = 48
# [c1] = 0.741


In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.tools import add_constant
from sklearn.feature_selection import SequentialFeatureSelector
import warnings
warnings.filterwarnings('ignore')

# 1. Load the data
url = "https://liangfgithub.github.io/Data/Caravan.csv"
data = pd.read_csv(url)

# Encode the response variable
data['Purchase'] = data['Purchase'].map({'No':0, 'Yes':1})

# Split into test and training
test_data = data.iloc[:1000]
train_data = data.iloc[1000:]

X_train = train_data.drop('Purchase', axis=1)
y_train = train_data['Purchase']
X_test = test_data.drop('Purchase', axis=1)
y_test = test_data['Purchase']

# Standardize the predictors
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Function to calculate misclassifications and AUC
def evaluate_model(model, X, y, cutoff=0.25):
    probs = model.predict_proba(X)[:,1]
    preds = (probs >= cutoff).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, preds).ravel()
    auc = roc_auc_score(y, probs)
    return fp, fn, auc

### **Question [a1], [b1], [c1]: Logistic Regression with All Predictors**

# Fit logistic regression with all predictors
log_reg = LogisticRegression(penalty=None,max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Evaluate on test data
a1, b1, c1 = evaluate_model(log_reg, X_test_scaled, y_test)

print(f"[a1] = {a1}")
print(f"[b1] = {b1}")
print(f"[c1] = {c1:.3f}")

### **Forward Variable Selection Using AIC**

# Add constant for statsmodels
X_train_sm = add_constant(X_train)
X_test_sm = add_constant(X_test)

# Fit model using statsmodels to perform forward selection based on AIC
def forward_selection_aic(X, y):
    cols = list(X.columns)
    selected = []
    current_score, best_new_score = np.inf, np.inf
    while cols:
        scores_with_candidates = []
        for candidate in cols:
            model = sm.Logit(y, X[selected + [candidate]]).fit(disp=0, method='bfgs')
            aic = model.aic
            scores_with_candidates.append((aic, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        if best_new_score < current_score:
            selected.append(best_candidate)
            current_score = best_new_score
            cols.remove(best_candidate)
        else:
            break
    return selected

selected_aic = forward_selection_aic(X_train_sm.drop('const', axis=1), y_train)
d2 = len(selected_aic)

# Fit logistic regression with selected features
log_reg_aic = LogisticRegression(max_iter=1000, penalty=None)
log_reg_aic.fit(X_train_scaled[:, [X_train.columns.get_loc(col) for col in selected_aic]], y_train)

# Prepare test data
X_test_aic = X_test_scaled[:, [X_train.columns.get_loc(col) for col in selected_aic]]

# Evaluate
a2, b2, c2 = evaluate_model(log_reg_aic, X_test_aic, y_test)

print(f"[d2] = {d2}")
print(f"[a2] = {a2}")
print(f"[b2] = {b2}")
print(f"[c2] = {c2:.3f}")

### **Forward Variable Selection Using BIC**

# Fit model using statsmodels to perform forward selection based on BIC
def forward_selection_bic(X, y):
    cols = list(X.columns)
    selected = []
    current_bic, best_new_bic = np.inf, np.inf
    while cols:
        scores_with_candidates = []
        for candidate in cols:
            model = sm.Logit(y, X[selected + [candidate]]).fit(disp=0)
            bic = model.bic
            scores_with_candidates.append((bic, candidate))
        scores_with_candidates.sort()
        best_new_bic, best_candidate = scores_with_candidates[0]
        if best_new_bic < current_bic:
            selected.append(best_candidate)
            current_bic = best_new_bic
            cols.remove(best_candidate)
        else:
            break
    return selected

selected_bic = forward_selection_bic(X_train_sm.drop('const', axis=1), y_train)
d3 = len(selected_bic)

# Fit logistic regression with selected features
log_reg_bic = LogisticRegression(max_iter=1000)
log_reg_bic.fit(X_train_scaled[:, [X_train.columns.get_loc(col) for col in selected_bic]], y_train)

# Prepare test data
X_test_bic = X_test_scaled[:, [X_train.columns.get_loc(col) for col in selected_bic]]

# Evaluate
a3, b3, c3 = evaluate_model(log_reg_bic, X_test_bic, y_test)

print(f"[d3] = {d3}")
print(f"[a3] = {a3}")
print(f"[b3] = {b3}")
print(f"[c3] = {c3:.3f}")

### **L1 Penalty (Lasso) Logistic Regression**

# Fit L1-penalized logistic regression
log_reg_l1 = LogisticRegression(penalty='l1', C=1/0.004, solver='liblinear', max_iter=1000)
log_reg_l1.fit(X_train_scaled, y_train)

# Number of non-intercept predictors (non-zero coefficients)
d4 = np.sum(log_reg_l1.coef_ != 0)

# Evaluate on test data
a4, b4, c4 = evaluate_model(log_reg_l1, X_test_scaled, y_test)

print(f"[d4] = {d4}")
print(f"[a4] = {a4}")
print(f"[b4] = {b4}")
print(f"[c4] = {c4:.3f}")


[a1] = 24
[b1] = 48
[c1] = 0.741
[d2] = 15
[a2] = 13
[b2] = 54
[c2] = 0.743


LinAlgError: Singular matrix